# 查询一：数据库基本情况查询

In [1]:
import sqlite3
import json
import os
from datetime import datetime
from typing import Optional
import pandas as pd


class NotebookDatabaseChecker:
    """适用于Jupyter Notebook的数据库检查器"""

    def __init__(self, db_path: str = "memory/memory.sqlite"):
        self.db_path = db_path
        self.conn = None

    def connect(self) -> bool:
        """连接数据库"""
        if not os.path.exists(self.db_path):
            print(f"❌ 数据库文件不存在: {self.db_path}")
            return False
        try:
            self.conn = sqlite3.connect(self.db_path)
            self.conn.row_factory = sqlite3.Row
            print(f"✅ 成功连接数据库: {self.db_path}")
            return True
        except Exception as e:
            print(f"❌ 连接数据库失败: {e}")
            return False

    def close(self):
        """关闭数据库连接"""
        if self.conn:
            self.conn.close()

    def get_table_info(self) -> pd.DataFrame:
        """获取数据库中所有表及结构信息"""
        cursor = self.conn.cursor()
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cursor.fetchall()
        all_info = []
        for table in tables:
            table_name = table["name"]
            cursor.execute(f"PRAGMA table_info({table_name});")
            columns = cursor.fetchall()
            for col in columns:
                all_info.append({
                    "表名": table_name,
                    "列名": col["name"],
                    "类型": col["type"],
                    "非空": bool(col["notnull"]),
                    "默认值": col["dflt_value"],
                    "主键": bool(col["pk"]),
                })
        return pd.DataFrame(all_info)

    def show_summary(self) -> pd.DataFrame:
        """显示数据库概要统计"""
        stats = []
        cursor = self.conn.cursor()

        cursor.execute("SELECT COUNT(*) FROM agents;")
        stats.append(("Agents", cursor.fetchone()[0]))

        cursor.execute("SELECT COUNT(*) FROM memories;")
        stats.append(("Memories", cursor.fetchone()[0]))

        cursor.execute("SELECT COUNT(*) FROM conversations;")
        stats.append(("Conversations", cursor.fetchone()[0]))

        cursor.execute("SELECT COUNT(*) FROM daily_plans;")
        stats.append(("Daily Plans", cursor.fetchone()[0]))

        # 显示plan类型的记忆数量（旧的计划数据）
        cursor.execute("SELECT COUNT(*) FROM memories WHERE memory_type = 'plan';")
        stats.append(("Plan Memories (旧计划)", cursor.fetchone()[0]))

        return pd.DataFrame(stats, columns=["表名", "记录数"])
    
    ## agents 表
    def get_agents(self) -> pd.DataFrame:
        """获取agents表数据"""
        query = "SELECT * FROM agents ORDER BY created_at;"
        return pd.read_sql_query(query, self.conn)
    
    ## memories 表 
    def get_memories(self, limit: int = 20, agent_id: Optional[str] = None) -> pd.DataFrame:
        """获取memories表数据"""
        if agent_id:
            query = f"""
                SELECT m.*, a.name AS agent_name
                FROM memories m
                LEFT JOIN agents a ON m.agent_id = a.id
                WHERE m.agent_id = ?
                ORDER BY m.timestamp DESC
                LIMIT ?
            """
            return pd.read_sql_query(query, self.conn, params=(agent_id, limit))
        else:
            query = f"""
                SELECT m.*, a.name AS agent_name
                FROM memories m
                LEFT JOIN agents a ON m.agent_id = a.id
                ORDER BY m.timestamp DESC
                LIMIT ?
            """
            return pd.read_sql_query(query, self.conn, params=(limit,))

    def get_conversations(self, limit: int = 20, agent_id: Optional[str] = None) -> pd.DataFrame:
        """获取conversations表数据"""
        if agent_id:
            query = f"""
                SELECT c.*, a1.name AS from_name, a2.name AS to_name
                FROM conversations c
                LEFT JOIN agents a1 ON c.from_agent = a1.id
                LEFT JOIN agents a2 ON c.to_agent = a2.id
                WHERE c.from_agent = ? OR c.to_agent = ?
                ORDER BY c.timestamp DESC
                LIMIT ?
            """
            return pd.read_sql_query(query, self.conn, params=(agent_id, agent_id, limit))
        else:
            query = f"""
                SELECT c.*, a1.name AS from_name, a2.name AS to_name
                FROM conversations c
                LEFT JOIN agents a1 ON c.from_agent = a1.id
                LEFT JOIN agents a2 ON c.to_agent = a2.id
                ORDER BY c.timestamp DESC
                LIMIT ?
            """
            return pd.read_sql_query(query, self.conn, params=(limit,))

    def get_daily_plans(self, limit: int = 20, agent_id: Optional[str] = None) -> pd.DataFrame:
        """获取daily_plans表数据（新的计划数据）"""
        if agent_id:
            query = f"""
                SELECT dp.*, a.name AS agent_name
                FROM daily_plans dp
                LEFT JOIN agents a ON dp.agent_id = a.id
                WHERE dp.agent_id = ?
                ORDER BY dp.created_at DESC
                LIMIT ?
            """
            return pd.read_sql_query(query, self.conn, params=(agent_id, limit))
        else:
            query = f"""
                SELECT dp.*, a.name AS agent_name
                FROM daily_plans dp
                LEFT JOIN agents a ON dp.agent_id = a.id
                ORDER BY dp.created_at DESC
                LIMIT ?
            """
            return pd.read_sql_query(query, self.conn, params=(limit,))

    def get_plan_memories(self, limit: int = 20, agent_id: Optional[str] = None) -> pd.DataFrame:
        """获取plan类型的记忆数据（旧的计划数据）"""
        if agent_id:
            query = f"""
                SELECT m.*, a.name AS agent_name
                FROM memories m
                LEFT JOIN agents a ON m.agent_id = a.id
                WHERE m.memory_type = 'plan' AND m.agent_id = ?
                ORDER BY m.timestamp DESC
                LIMIT ?
            """
            return pd.read_sql_query(query, self.conn, params=(agent_id, limit))
        else:
            query = f"""
                SELECT m.*, a.name AS agent_name
                FROM memories m
                LEFT JOIN agents a ON m.agent_id = a.id
                WHERE m.memory_type = 'plan'
                ORDER BY m.timestamp DESC
                LIMIT ?
            """
            return pd.read_sql_query(query, self.conn, params=(limit,))

    def analyze_plans(self) -> dict:
        """分析计划数据分布"""
        cursor = self.conn.cursor()
        
        # 统计daily_plans表中的数据
        cursor.execute("SELECT COUNT(*) FROM daily_plans;")
        daily_plans_count = cursor.fetchone()[0]
        
        # 统计memories表中plan类型的数据
        cursor.execute("SELECT COUNT(*) FROM memories WHERE memory_type = 'plan';")
        plan_memories_count = cursor.fetchone()[0]
        
        # 按agent统计daily_plans
        cursor.execute("""
            SELECT a.name, COUNT(*) as count
            FROM daily_plans dp
            LEFT JOIN agents a ON dp.agent_id = a.id
            GROUP BY dp.agent_id, a.name
            ORDER BY count DESC
        """)
        daily_plans_by_agent = cursor.fetchall()
        
        # 按agent统计plan memories
        cursor.execute("""
            SELECT a.name, COUNT(*) as count
            FROM memories m
            LEFT JOIN agents a ON m.agent_id = a.id
            WHERE m.memory_type = 'plan'
            GROUP BY m.agent_id, a.name
            ORDER BY count DESC
        """)
        plan_memories_by_agent = cursor.fetchall()
        
        return {
            "daily_plans_total": daily_plans_count,
            "plan_memories_total": plan_memories_count,
            "daily_plans_by_agent": [(row[0], row[1]) for row in daily_plans_by_agent],
            "plan_memories_by_agent": [(row[0], row[1]) for row in plan_memories_by_agent]
        }


# 初始化对象
db_checker = NotebookDatabaseChecker("/Users/brycewang/Documents/GitHub/MamaVillage/memory/memory.sqlite")
connected = db_checker.connect()

✅ 成功连接数据库: /Users/brycewang/Documents/GitHub/MamaVillage/memory/memory.sqlite


In [2]:
# 如果成功连接，则展示表结构和概要统计
table_info_df = db_checker.get_table_info()
summary_df = db_checker.show_summary()
summary_df 

,表名,记录数
0,Agents,4
1,Memories,96
2,Conversations,6
3,Daily Plans,1
4,Plan Memories (旧计划),40


In [3]:
# 获取所有 agents
df_agents = db_checker.get_agents()
df_agents 

,id,name,age,education,role,personality,created_at
0,mama_wangfang,王芳,32,大专,经验妈妈,"{""traits"": [""实用主义"", ""善于总结"", ""乐于助人"", ""理性冷静""], ""...",2025-07-13 20:57:24
1,mama_xiaochen,小陈,25,中专,新手妈妈,"{""traits"": [""谨慎小心"", ""容易担心"", ""虚心学习"", ""依赖性强""], ""...",2025-07-13 20:57:24
2,mama_xiaoli,小李,28,高中,年轻妈妈,"{""traits"": [""细心"", ""好学"", ""爱刷手机"", ""容易焦虑""], ""inte...",2025-07-13 20:57:24
3,grandma_zhang,张奶奶,58,初中,资深奶奶,"{""traits"": [""经验丰富"", ""热心肠"", ""传统智慧"", ""学习新事物""], ""...",2025-07-13 20:57:24


In [4]:
# 获取最近 20 条记忆
df_memories = db_checker.get_memories()
df_memories 

,id,agent_id,memory_type,content,importance,timestamp,metadata,agent_name
0,96,mama_xiaochen,reflection,今天最重要的事情就是给妞妞准备辅食，看到她吃得开心，我心里也跟着舒坦。通过学习疫苗知识，我觉...,7,2025-07-13 20:59:42,"{""reflection_date"": ""2025-07-13"", ""memories_co...",小陈
1,94,grandma_zhang,reflection,今天最重要的事情就是和村里的妈妈们聚在一起，聊聊孩子们的学习和生活，心里暖暖的。我学到了不少...,7,2025-07-13 20:59:41,"{""reflection_date"": ""2025-07-13"", ""memories_co...",张奶奶
2,95,mama_xiaochen,daily_activity,育儿活动：给孩子准备辅食,3,2025-07-13 20:59:41,"{""scenario"": ""给孩子准备辅食""}",小陈
3,93,grandma_zhang,action,进行了社交互动：private_message,4,2025-07-13 20:59:39,"{""action"": {""action"": ""在群里聊天交流"", ""type"": ""soci...",张奶奶
4,92,mama_xiaoli,reflection,今天最重要的事情就是照顾小宝了，他有点不舒服，我心里真是担心得很。虽然我准备了营养辅食，还带...,7,2025-07-13 20:59:38,"{""reflection_date"": ""2025-07-13"", ""memories_co...",小李
5,91,mama_xiaoli,action,进行了社交互动：private_message,4,2025-07-13 20:59:36,"{""action"": {""action"": ""在群里聊天交流"", ""type"": ""soci...",小李
6,86,mama_xiaoli,plan,计划行动：准备营养午餐,9,2025-07-13 20:59:35,"{""action"": ""准备营养午餐"", ""priority"": 9, ""descripti...",小李
7,87,mama_xiaoli,plan,计划行动：带小宝去公园散步,8,2025-07-13 20:59:35,"{""action"": ""带小宝去公园散步"", ""priority"": 8, ""descrip...",小李
8,88,mama_xiaoli,plan,计划行动：观看育儿视频,7,2025-07-13 20:59:35,"{""action"": ""观看育儿视频"", ""priority"": 7, ""descripti...",小李
9,89,mama_xiaoli,plan,计划行动：和小宝一起做手工,6,2025-07-13 20:59:35,"{""action"": ""和小宝一起做手工"", ""priority"": 6, ""descrip...",小李


In [5]:
# 获取最近 20 条对话
df_convs = db_checker.get_conversations()
df_convs 

,id,from_agent,to_agent,message,conversation_type,timestamp,metadata,from_name,to_name
0,6,grandma_zhang,mama_xiaoli,孩子啊，我跟你说，辅食真是个头疼事，网上的乱七八糟的我都看不懂😅。不过跟村里其他妈妈聊聊，学...,private_chat,2025-07-13 20:59:39,"{""type"": ""private_chat"", ""to_agent"": ""mama_xia...",张奶奶,小李
1,5,mama_xiaoli,mama_wangfang,哎呀，今天准备给小宝做营养午餐呢！你们都给孩子做什么呀？😊想带他去公园散步，顺便看看育儿视频...,private_chat,2025-07-13 20:59:36,"{""type"": ""private_chat"", ""to_agent"": ""mama_wan...",小李,王芳
2,4,mama_xiaochen,grandma_zhang,小陈：唉，最近给宝宝准备辅食，感觉好紧张啊😰，不知道吃什么比较好？你们都是怎么弄的？还有，宝...,private_chat,2025-07-13 20:59:04,"{""type"": ""private_chat"", ""to_agent"": ""grandma_...",小陈,张奶奶
3,3,grandma_zhang,None,孩子们，家庭和谐真重要啊！❤️ 我跟你们说，平时多沟通，别总是忙工作，陪陪孩子吃顿饭，心里就...,group_chat,2025-07-13 20:59:00,"{""type"": ""group_chat"", ""topic"": ""家庭和谐"", ""motiv...",张奶奶,None
4,2,mama_xiaoli,None,哎呀，亲爱的们，我最近看了一个育儿视频，教了不少预防感冒的小技巧，真的是很有用哦！😘💕大家可...,content_sharing,2025-07-13 20:58:27,"{""type"": ""content_sharing"", ""content"": {""title...",小李,None
5,1,mama_xiaochen,None,哎呀，大家好，我是小陈，最近打算给妞妞做辅食，感觉有点紧张，不知道该怎么开始？🤔 还有，想带...,help_request,2025-07-13 20:58:23,"{""type"": ""help_request"", ""scenario"": ""孩子有点不舒服""...",小陈,None


In [6]:
# 获取最近 20 条每日计划
df_plans = db_checker.get_daily_plans()
df_plans 

,id,agent_id,planned_action,priority,status,planned_time,created_at,agent_name
0,1,test_agent,测试计划,5,pending,上午,2025-07-13 21:22:42,None


# 查询二：关键词查询

In [7]:
def search_keyword_in_memories(conn, keyword: str, limit: int = 50):
    query = """
        SELECT m.*, a.name AS agent_name
        FROM memories m
        LEFT JOIN agents a ON m.agent_id = a.id
        WHERE m.content LIKE ?
        ORDER BY m.timestamp DESC
        LIMIT ?
    """
    return pd.read_sql_query(query, conn, params=(f'%{keyword}%', limit))

def search_keyword_in_conversations(conn, keyword: str, limit: int = 50):
    query = """
        SELECT c.*, a1.name AS from_name, a2.name AS to_name
        FROM conversations c
        LEFT JOIN agents a1 ON c.from_agent = a1.id
        LEFT JOIN agents a2 ON c.to_agent = a2.id
        WHERE c.message LIKE ?
        ORDER BY c.timestamp DESC
        LIMIT ?
    """
    return pd.read_sql_query(query, conn, params=(f'%{keyword}%', limit))


In [8]:
keyword = "辅食"
search_keyword_in_memories(db_checker.conn, keyword)

,id,agent_id,memory_type,content,importance,timestamp,metadata,agent_name
0,96,mama_xiaochen,reflection,今天最重要的事情就是给妞妞准备辅食，看到她吃得开心，我心里也跟着舒坦。通过学习疫苗知识，我觉...,7,2025-07-13 20:59:42,"{""reflection_date"": ""2025-07-13"", ""memories_co...",小陈
1,94,grandma_zhang,reflection,今天最重要的事情就是和村里的妈妈们聚在一起，聊聊孩子们的学习和生活，心里暖暖的。我学到了不少...,7,2025-07-13 20:59:41,"{""reflection_date"": ""2025-07-13"", ""memories_co...",张奶奶
2,95,mama_xiaochen,daily_activity,育儿活动：给孩子准备辅食,3,2025-07-13 20:59:41,"{""scenario"": ""给孩子准备辅食""}",小陈
3,92,mama_xiaoli,reflection,今天最重要的事情就是照顾小宝了，他有点不舒服，我心里真是担心得很。虽然我准备了营养辅食，还带...,7,2025-07-13 20:59:38,"{""reflection_date"": ""2025-07-13"", ""memories_co...",小李
4,83,mama_xiaochen,reflection,今天最重要的事情就是给妞妞准备辅食，看着她吃得开心，我心里也跟着舒坦。通过学习疫苗知识，我感...,7,2025-07-13 20:59:21,"{""reflection_date"": ""2025-07-13"", ""memories_co...",小陈
5,77,mama_xiaochen,plan,计划行动：准备辅食,9,2025-07-13 20:59:19,"{""action"": ""准备辅食"", ""priority"": 9, ""description...",小陈
6,68,mama_xiaochen,reflection,今天最重要的事情是给妞妞准备辅食，看到她吃得开心，我心里也舒坦了。通过学习疫苗知识，我明白了...,7,2025-07-13 20:59:05,"{""reflection_date"": ""2025-07-13"", ""memories_co...",小陈
7,62,grandma_zhang,reflection,今天最重要的事情就是和村里的其他妈妈们聊聊，大家一起关心孩子的学习和生活，心里特别暖和。我学...,7,2025-07-13 20:58:54,"{""reflection_date"": ""2025-07-13"", ""memories_co...",张奶奶
8,63,mama_xiaochen,daily_activity,育儿活动：给孩子准备辅食,3,2025-07-13 20:58:54,"{""scenario"": ""给孩子准备辅食""}",小陈
9,52,mama_xiaoli,reflection,今天最重要的事情是照顾小宝，他有点不舒服，我心里很担心。虽然我准备了营养辅食，还带他去公园散...,7,2025-07-13 20:58:42,"{""reflection_date"": ""2025-07-13"", ""memories_co...",小李


In [9]:
keyword = "孩子"
search_keyword_in_conversations(db_checker.conn, keyword)

,id,from_agent,to_agent,message,conversation_type,timestamp,metadata,from_name,to_name
0,6,grandma_zhang,mama_xiaoli,孩子啊，我跟你说，辅食真是个头疼事，网上的乱七八糟的我都看不懂😅。不过跟村里其他妈妈聊聊，学...,private_chat,2025-07-13 20:59:39,"{""type"": ""private_chat"", ""to_agent"": ""mama_xia...",张奶奶,小李
1,5,mama_xiaoli,mama_wangfang,哎呀，今天准备给小宝做营养午餐呢！你们都给孩子做什么呀？😊想带他去公园散步，顺便看看育儿视频...,private_chat,2025-07-13 20:59:36,"{""type"": ""private_chat"", ""to_agent"": ""mama_wan...",小李,王芳
2,3,grandma_zhang,None,孩子们，家庭和谐真重要啊！❤️ 我跟你们说，平时多沟通，别总是忙工作，陪陪孩子吃顿饭，心里就...,group_chat,2025-07-13 20:59:00,"{""type"": ""group_chat"", ""topic"": ""家庭和谐"", ""motiv...",张奶奶,None
3,2,mama_xiaoli,None,哎呀，亲爱的们，我最近看了一个育儿视频，教了不少预防感冒的小技巧，真的是很有用哦！😘💕大家可...,content_sharing,2025-07-13 20:58:27,"{""type"": ""content_sharing"", ""content"": {""title...",小李,None


# 查询三：Daily Plans 分析（新增功能）

这个部分用于分析daily plans的保存情况，包括新的daily_plans表和旧的plan类型记忆。

In [10]:
# 分析计划数据分布
plan_analysis = db_checker.analyze_plans()
print("📊 计划数据分析报告")
print("=" * 50)
print(f"Daily Plans表中的记录数: {plan_analysis['daily_plans_total']}")
print(f"Plan类型记忆中的记录数: {plan_analysis['plan_memories_total']}")
print()

print("📋 Daily Plans表 - 按Agent分布:")
for agent_name, count in plan_analysis['daily_plans_by_agent']:
    print(f"  {agent_name}: {count} 条")

print()
print("🧠 Plan类型记忆 - 按Agent分布:")
for agent_name, count in plan_analysis['plan_memories_by_agent']:
    print(f"  {agent_name}: {count} 条")

if plan_analysis['daily_plans_total'] == 0 and plan_analysis['plan_memories_total'] > 0:
    print()
    print("💡 发现问题:")
    print("- Daily Plans表中没有记录，但Plan类型记忆中有记录")
    print("- 这说明之前的计划保存在了memories表中，而不是daily_plans表")
    print("- 已修复代码，新的模拟将正确保存到daily_plans表")

📊 计划数据分析报告
Daily Plans表中的记录数: 1
Plan类型记忆中的记录数: 40

📋 Daily Plans表 - 按Agent分布:
  None: 1 条

🧠 Plan类型记忆 - 按Agent分布:
  王芳: 15 条
  小陈: 10 条
  小李: 10 条
  张奶奶: 5 条


In [11]:
# 查看旧的计划数据（保存在memories表中的plan类型记忆）
print("🧠 旧的计划数据（来自memories表）:")
df_old_plans = db_checker.get_plan_memories(limit=50)
df_old_plans.head(10)

🧠 旧的计划数据（来自memories表）:


,id,agent_id,memory_type,content,importance,timestamp,metadata,agent_name
0,86,mama_xiaoli,plan,计划行动：准备营养午餐,9,2025-07-13 20:59:35,"{""action"": ""准备营养午餐"", ""priority"": 9, ""descripti...",小李
1,87,mama_xiaoli,plan,计划行动：带小宝去公园散步,8,2025-07-13 20:59:35,"{""action"": ""带小宝去公园散步"", ""priority"": 8, ""descrip...",小李
2,88,mama_xiaoli,plan,计划行动：观看育儿视频,7,2025-07-13 20:59:35,"{""action"": ""观看育儿视频"", ""priority"": 7, ""descripti...",小李
3,89,mama_xiaoli,plan,计划行动：和小宝一起做手工,6,2025-07-13 20:59:35,"{""action"": ""和小宝一起做手工"", ""priority"": 6, ""descrip...",小李
4,90,mama_xiaoli,plan,计划行动：和朋友视频聊天,5,2025-07-13 20:59:35,"{""action"": ""和朋友视频聊天"", ""priority"": 5, ""descript...",小李
5,77,mama_xiaochen,plan,计划行动：准备辅食,9,2025-07-13 20:59:19,"{""action"": ""准备辅食"", ""priority"": 9, ""description...",小陈
6,78,mama_xiaochen,plan,计划行动：宝宝的午睡时间,8,2025-07-13 20:59:19,"{""action"": ""宝宝的午睡时间"", ""priority"": 8, ""descript...",小陈
7,79,mama_xiaochen,plan,计划行动：查看育儿视频,7,2025-07-13 20:59:19,"{""action"": ""查看育儿视频"", ""priority"": 7, ""descripti...",小陈
8,80,mama_xiaochen,plan,计划行动：与邻居或朋友聊天,6,2025-07-13 20:59:19,"{""action"": ""与邻居或朋友聊天"", ""priority"": 6, ""descrip...",小陈
9,81,mama_xiaochen,plan,计划行动：准备晚餐,8,2025-07-13 20:59:19,"{""action"": ""准备晚餐"", ""priority"": 8, ""description...",小陈


In [12]:
# 查看具体的计划内容
print("📋 各Agent的计划分布详情:")
print()

agents = ['mama_xiaoli', 'mama_wangfang', 'mama_xiaochen', 'grandma_zhang']
agent_names = {'mama_xiaoli': '小李', 'mama_wangfang': '王芳', 'mama_xiaochen': '小陈', 'grandma_zhang': '张奶奶'}

for agent_id in agents:
    agent_plans = db_checker.get_plan_memories(limit=20, agent_id=agent_id)
    agent_name = agent_names[agent_id]
    
    print(f"👤 {agent_name} ({agent_id}): {len(agent_plans)} 条计划")
    
    if len(agent_plans) > 0:
        print("   最近的计划:")
        for i, plan in agent_plans.head(5).iterrows():
            content = plan['content']
            timestamp = plan['timestamp'][:16]  # 只显示到分钟
            importance = plan['importance']
            print(f"   {i+1}. [{timestamp}] {content} (重要性: {importance})")
        
        if len(agent_plans) > 5:
            print(f"   ... 还有 {len(agent_plans) - 5} 条计划")
    else:
        print("   暂无计划记录")
    
    print()

📋 各Agent的计划分布详情:

👤 小李 (mama_xiaoli): 10 条计划
   最近的计划:
   1. [2025-07-13 20:59] 计划行动：准备营养午餐 (重要性: 9)
   2. [2025-07-13 20:59] 计划行动：带小宝去公园散步 (重要性: 8)
   3. [2025-07-13 20:59] 计划行动：观看育儿视频 (重要性: 7)
   4. [2025-07-13 20:59] 计划行动：和小宝一起做手工 (重要性: 6)
   5. [2025-07-13 20:59] 计划行动：和朋友视频聊天 (重要性: 5)
   ... 还有 5 条计划

👤 王芳 (mama_wangfang): 15 条计划
   最近的计划:
   1. [2025-07-13 20:59] 计划行动：陪大宝和二宝一起做手工 (重要性: 8)
   2. [2025-07-13 20:59] 计划行动：与孩子们共进晚餐 (重要性: 9)
   3. [2025-07-13 20:59] 计划行动：观看育儿视频，学习健康小知识 (重要性: 7)
   4. [2025-07-13 20:59] 计划行动：带孩子们去公园散步 (重要性: 6)
   5. [2025-07-13 20:59] 计划行动：安排孩子的学习时间 (重要性: 8)
   ... 还有 10 条计划

👤 小陈 (mama_xiaochen): 10 条计划
   最近的计划:
   1. [2025-07-13 20:59] 计划行动：准备辅食 (重要性: 9)
   2. [2025-07-13 20:59] 计划行动：宝宝的午睡时间 (重要性: 8)
   3. [2025-07-13 20:59] 计划行动：查看育儿视频 (重要性: 7)
   4. [2025-07-13 20:59] 计划行动：与邻居或朋友聊天 (重要性: 6)
   5. [2025-07-13 20:59] 计划行动：准备晚餐 (重要性: 8)
   ... 还有 5 条计划

👤 张奶奶 (grandma_zhang): 5 条计划
   最近的计划:
   1. [2025-07-13 20:58] 计划行动：陪孙子孙女玩游戏 (重要性: 8)
   2. [2025-0

In [15]:
# END